In [32]:
%load_ext autoreload
%autoreload 2
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
from IPython.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
raw_feature = read_raw_feature()
label_material = read_material_label()
groups_VSP, group_material = read_VSP_group()

N = raw_feature.shape[0]

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
def make_image_html(image_path):
     return '<img src="{}" width="100" height="100" style="display:inline;margin:1px"/>'.format(image_path)

In [39]:
label_visiprog = get_VSP_label(groups_VSP, N)

In [40]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [41]:
# accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp = []

list_img = read_img_list()

n_cluster_list = [2,5,8]
n_cluster = 3

for i in range(2):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                     test_size=0.2, stratify=label_material)
    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_train = model.transform(X_train)
        X_mat_test = model.transform(X_test)
        res = kmean_subclass(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test, n_cluster)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_train = model_vsp.transform(X_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = kmean_subclass(X_vsp_train, Y_mat_train, X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)
        

INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[41 13 12 ...,  7 45 20]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[41 13 12 ...,  7 45 20]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[19 10 43 ..., 26 21  2]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[19 10 43 ..., 26 21  2]
INFO:visiprog.evaluate:Running on test set


In [43]:
# find out where does semantic perform better than VSP
outperform = ((res['prediction'] == Y_mat_test) & (res_vsp['prediction'] != Y_mat_test))
idx_outperform = np.where(outperform == True)[0]

display(HTML(''.join(make_image_html(list_img[i]) for i in idx_outperform)))

for i in idx_outperform:
    print("image {}, GT: {}, prediction {}".format(os.path.basename(list_img[i]), label_material[i], \
                                                   int(res_vsp['prediction'][i])))

image 01-063.png, GT: 0, prediction 27
image 01-096.png, GT: 0, prediction 44
image 02-026.png, GT: 1, prediction 48
image 02-033.png, GT: 1, prediction 19
image 02-036.png, GT: 1, prediction 2
image 02-051.png, GT: 1, prediction 42
image 02-078.png, GT: 1, prediction 57
image 02-129.png, GT: 1, prediction 27
image 03-016.png, GT: 2, prediction 49
image 03-068.png, GT: 2, prediction 58
image 03-090.png, GT: 2, prediction 19
image 03-175.png, GT: 2, prediction 38
image 03-179.png, GT: 2, prediction 0
image 04-002.png, GT: 3, prediction 11
image 04-030.png, GT: 3, prediction 51
image 04-059.png, GT: 3, prediction 51
image 04-134.png, GT: 3, prediction 21
image 04-141.png, GT: 3, prediction 7
image 05-061m.png, GT: 4, prediction 5
image 05-104m.png, GT: 4, prediction 10
image 05-116m.png, GT: 4, prediction 0
image 05-143m.png, GT: 4, prediction 42
image 06-079.png, GT: 5, prediction 0
image 06-145.png, GT: 5, prediction 52
image 06-150.png, GT: 5, prediction 2
image 07-033.png, GT: 6, pre

In [42]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat}, columns=['visiprog','material'])
df.loc['average'] = df.mean()
display(df)

,visiprog,material
0,0.906578,0.949476
1,0.907531,0.948522
average,0.907054,0.948999


In [7]:
print(Y_mat_test)

[ 8 60 18 ..., 36 55  1]
